In [18]:
import numpy as np
import scipy
import sys, os
sys.path.insert(0, os.path.abspath(".."))
import matplotlib.pyplot as plt
np.set_printoptions(precision=5, suppress=True, linewidth=100)
plt.rcParams['figure.dpi'] = 150
import tenpy
import tenpy.linalg.np_conserved as npc
from tenpy.networks.mps import MPS
from tenpy.models.model import CouplingMPOModel
from tenpy.networks import site
import matplotlib.pyplot as plt
tenpy.tools.misc.setup_logging(to_stdout="INFO")
import time
import csv
import pandas as pd
from tenpy.models.lattice import Square
from numpy.linalg import svd
from src.MPO_functions import MPS_drop_charge, double_to_mpo, mpo_product, mpo_trace, truncate_mpo

from tenpy.models.lattice import Triangular
import tenpy.linalg.np_conserved as npc
from tenpy.algorithms import dmrg
import tenpy.linalg.np_conserved as npc
from tenpy.linalg.truncation import svd_theta
from tenpy.linalg.sparse import FlatLinearOperator
from tenpy.tools.math import speigs
import copy

### Two copy Ising model (Z2 symmetric Brownian circuit in the doubled HIlbert space)

In [4]:
class Ising_2copy(CouplingMPOModel):
    """Average time evolution in doubled Hilbert space of random TFI"""

    default_lattice = "Chain"
    force_default_lattice = True

    def init_sites(self, model_params):
        X= np.array([[0,1],[1,0]])
        spin1 = site.SpinHalfSite(conserve="parity")
        spin2 = site.SpinHalfSite(conserve="parity")
        #spin1.add_op('Sx',1/2*X)
        #spin2.add_op('Sx',1/2*X)
        sites = [spin1, spin2]
        
        site.set_common_charges(sites, new_charges='independent')
        return [spin1, spin2], ['sf', 'sb']  # return species names as well!
        # this causes `init_lattice` to initialize a `MultiSpeciesLattice`
        
    def init_terms(self, model_params):
        J = model_params.get('J', 1.)
        U = model_params.get('U', 1.)
        s = model_params.get('s', 0.5)
        p = model_params.get('p', 0.1)
        # onsite f-b interaction
        self.add_coupling(-2*U, 0, 'Sigmaz', 1, 'Sigmaz', [0])
        # magnetic field from postselection
        self.add_onsite(-p*s/4,0,'Sigmaz')
        self.add_onsite(-p*s/4,1,'Sigmaz')
        # f-b hopping term
        self.add_multi_coupling(-2*J,[('Sigmax',[0],0),('Sigmax',[1],0),('Sigmax',[0],1),('Sigmax',[1],1)])


### Run DMRG ground state search

In [ ]:
product_state = [('up','up')]
print('product_state = ')
print(product_state)
print()
initial_time=time.time()
Ls=[32]
Np=20

cor=np.zeros((len(Ls),Np)) #Renyi-2 correlators

ps=np.array([0.9,1.01,1.1])*8 #measurement rates

states=[]#ground states
for i,L in enumerate(Ls):
    print("L=",L)
    print()
    for j,prob in enumerate(ps):
        print("p=",prob)
        M=Ising_2copy({'L':L,'p':prob,'s':0.5})
        psi0 = MPS.from_lat_product_state(M.lat, product_state)
        eng = tenpy.algorithms.dmrg.SingleSiteDMRGEngine(psi0, M, {'mixer_params': {"amplitude": 1e-5},
                                                                        'trunc_params': {'chi_max': 100, 'svd_min':1e-7}, 'max_sweeps':30,'max_E_err':1.0e-12})
        E, psi = eng.run()
        #psi.test_sanity()
        #psi0 = MPS.from_lat_product_state(M.lat, product_state)
        #print("overlap with initial state=",psi.overlap(psi0))
        #print("norm**2=",psi.overlap(psi))
        #print(psi.entanglement_entropy())
        states.append(psi.copy())
        cor[i,j]=float(psi.expectation_value_term([('Sigmax',int((L/4-1)*2)),('Sigmax',int((L/4-1)*2+1)),('Sigmax',int(L-L/4-1)*2),('Sigmax',int(L-L/4-1)*2+1)]))
    print()
    print("---------------------------------------------------")
    print(time.time()-initial_time)

product_state = 
[('up', 'up')]

L= 32

p= 7.2
INFO    : Ising_2copy: reading 'L'=32
INFO    : Ising_2copy: reading 's'=0.5
INFO    : Ising_2copy: reading 'p'=np.float64(7.2)
INFO    : SingleSiteDMRGEngine: subconfig 'trunc_params'=Config(<2 options>, 'trunc_params')
INFO    : SingleSiteDMRGEngine: subconfig 'mixer_params'=Config(<1 options>, 'mixer_params')
INFO    : activate SubspaceExpansion with initial amplitude 1e-05
INFO    : SingleSiteDMRGEngine: reading 'max_sweeps'=30
INFO    : Running sweep with optimization
INFO    : trunc_params: reading 'chi_max'=100
INFO    : trunc_params: reading 'svd_min'=1e-07
INFO    : checkpoint after sweep 1
energy=-140.5824624420928615, max S=0.4731391314841594, age=64, norm_err=3.7e-01
Current memory usage 196.3MB, wall time: 1.1s
Delta E = nan, Delta S = 3.5915e-01 (per sweep)
max trunc_err = 3.9773e-35, max E_trunc = 1.2635e-02
chi: 3
INFO    : Running sweep with optimization
INFO    : checkpoint after sweep 2
energy=-140.7765208776160932, max 

KeyboardInterrupt: 